<a href="https://colab.research.google.com/github/ABOUSSALIM/ControleGSP/blob/master/flux_image_to_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title ##**Install** { display-mode: "form" }
%%capture
%cd /content
!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install torch
!pip install torchsde

import torch
import torchvision
import torchaudio
!apt -y install -qq aria2
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-dev-fp8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

!wget https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/test.png -O /content/test.png

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_post_processing
from totoro import model_management

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
VAEEncode = NODE_CLASS_MAPPINGS["VAEEncode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
ImageScaleToTotalPixels = nodes_post_processing.NODE_CLASS_MAPPINGS["ImageScaleToTotalPixels"]()

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

In [ ]:
#@title ##**Upload image** { display-mode: "form" }

from google.colab import files
from IPython.display import Image as IPythonImage
import io

uploaded = files.upload()

for filename in uploaded.keys():
    with open('/content/input.png', 'wb') as f:
        f.write(uploaded[filename])


In [ ]:
#@title ##**Run** { display-mode: "form" }
from PIL import Image
import random
import numpy as np
import torch
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

def resize_and_pad_to_nearest_multiple(image, multiple):
    width, height = image.size
    new_width = ((width - 1) // multiple + 1) * multiple
    new_height = ((height - 1) // multiple + 1) * multiple
    image = image.copy()
    image.thumbnail((new_width, new_height), Image.LANCZOS)
    new_image = Image.new('RGB', (new_width, new_height))
    left = (new_width - image.width) // 2
    top = (new_height - image.height) // 2
    new_image.paste(image, (left, top))
    return new_image, (left, top, image.width, image.height)

with torch.inference_mode():
    positive_prompt = "photo" #@param {type:"string"}
    seed = 0 #@param {type:"integer"}
    steps = 30 #@param {type:"slider", min:1, max:100, step:1}
    sampler_name = "euler" #@param ["euler", "ddim", "heun", "euler_cfg_pp", "euler_ancestral", "euler_ancestral_cfg_pp", "heunpp2", "dpm_2", "dpm_2_ancestral", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "dpmpp_3m_sde", "dpmpp_3m_sde_gpu", "ddpm", "Icm", "ipndm", "ipndm_v", "deis", "uni_pc", "uni_pc_bh2"]
    scheduler = "simple" #@param ["simple", "karras", "normal", "exponential", "sgm_uniform", "ddim_uniform", "beta"]
    sigma_value = 0.35 #@param {type:"slider", min:0.0, max:1.0, step:0.01}

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(f"seed: {seed}")

    cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, sigma_value)[0]

    image = Image.open("/content/input.png")
    image, padding_info = resize_and_pad_to_nearest_multiple(image, 64)
    width, height = image.size

    image.save("/content/resized_input.png")
    if torch.cuda.is_available():
      torch.cuda.empty_cache()
    gc.collect()

    image = nodes.LoadImage().load_image("/content/resized_input.png")[0]
    latent_image = VAEEncode.encode(vae, image)[0]

    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()

    result_image_path = "/content/flux.png"
    decoded_image = Image.fromarray(np.array(decoded * 255, dtype=np.uint8)[0])

    left, top, width, height = padding_info
    cropped_image = decoded_image.crop((left, top, left + width, top + height))
    cropped_image.save(result_image_path)

    from IPython.display import Image as IPythonImage
    display(IPythonImage(filename=result_image_path))
    if torch.cuda.is_available():
      torch.cuda.empty_cache()
    gc.collect()
